In [2]:
import chainlit as cl
from llama_deploy import LlamaDeployClient, ControlPlaneConfig#, SessionClient
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get configuration from environment variables
CONTROL_PLANE_HOST = "localhost"
CONTROL_PLANE_PORT = 8000
WORKFLOW_NAME = "react_workflow"

if not all([CONTROL_PLANE_HOST, WORKFLOW_NAME]):
    raise ValueError("Missing required environment variables. Please check your .env file.")

# Create a LlamaDeployClient instance
client = LlamaDeployClient(
    ControlPlaneConfig(
        host=CONTROL_PLANE_HOST,
        port=CONTROL_PLANE_PORT
    )
)

@cl.on_chat_start
async def start():
    """Initialize the chat session."""
    # Create a SessionClient instance and store it in the user session
    session = client.create_session()
    cl.user_session.set("llama_session", session)
    
    await cl.Message(
        content="👋 Hello! I'm your AI assistant powered by LlamaIndex. I can help you analyze "
        "Uber and Lyft's financial data from their 2021 10-K reports. What would you like to know?"
    ).send()

@cl.on_message
async def main(message: cl.Message):
    """Handle incoming messages."""
    try:
        # Get the SessionClient instance from the user session
        session = cl.user_session.get("llama_session")
        if not session:
            # If session is not found, create a new one
            session = client.create_session()
            cl.user_session.set("llama_session", session)

        # Create a thinking message
        thinking_msg = cl.Message(content="Thinking...")
        await thinking_msg.send()

        # Run the workflow and get the task result
        result = session.run(
            WORKFLOW_NAME,
            input=message.content
        )

        # Extract the response and sources
        print(f"result:{result}")
        final_response = result.get("response", "No response generated")
        sources = result.get("sources", [])

        # Create elements for the final message
        elements = []
        if result.get("current_reasoning"):
            reasoning_text = "\n".join(
                f"{step_num}. Thought: {step.thought}\nAction: {step.action}\nObservation: {step.observation}"
                for step_num, step in enumerate(result["current_reasoning"], 1)
            )
            elements.append(
                cl.Text(name="reasoning", content=reasoning_text, display="inline")
            )
        
        if sources:
            source_text = "\nSources:\n" + "\n".join(
                [f"- {source}" for source in sources]
            )
            elements.append(
                cl.Text(name="sources", content=source_text, display="inline")
            )

        # Send the final response
        await cl.Message(
            content=final_response,
            elements=elements
        ).send()

        # Remove the thinking message
        await thinking_msg.remove()

    except Exception as e:
        # Send error message
        await cl.Message(
            content=f"❌ An error occurred: {str(e)}"
        ).send()

if __name__ == "__main__":
    cl.run()

KeyError: 'run'

In [4]:
import chainlit as cl
from llama_deploy import LlamaDeployClient, ControlPlaneConfig#, SessionClient
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get configuration from environment variables
CONTROL_PLANE_HOST = "localhost"
CONTROL_PLANE_PORT = 8000
WORKFLOW_NAME = "react_workflow"

if not all([CONTROL_PLANE_HOST, WORKFLOW_NAME]):
    raise ValueError("Missing required environment variables. Please check your .env file.")

# Create a LlamaDeployClient instance
client = LlamaDeployClient(
    ControlPlaneConfig(
        host=CONTROL_PLANE_HOST,
        port=CONTROL_PLANE_PORT
    )
)

@cl.on_chat_start
async def start():
    """Initialize the chat session."""
    # Create a SessionClient instance and store it in the user session
    session = client.create_session()
    cl.user_session.set("llama_session", session)
    
    await cl.Message(
        content="👋 Hello! I'm your AI assistant powered by LlamaIndex. I can help you analyze "
        "Uber and Lyft's financial data from their 2021 10-K reports. What would you like to know?"
    ).send()

@cl.on_message
async def main(message: cl.Message={'role': 'user', 'content': 'Hello'}):
    """Handle incoming messages."""
    # Get the SessionClient instance from the user session
    session = cl.user_session.get("llama_session")
    if not session:
        # If session is not found, create a new one
        session = client.create_session()
        cl.user_session.set("llama_session", session)

    # Create a thinking message
    thinking_msg = cl.Message(content="Thinking...")
    await thinking_msg.send()

    # Run the workflow and get the task result
    result = session.run(
        WORKFLOW_NAME,
        input=message.content
    )
    print(result)

if __name__ == "__main__":
    cl.run()

KeyError: 'run'